# Record Linkage

In [262]:
import recordlinkage
import pandas as pd
import json
import collections

In [263]:
PATH_DS1 = './Mediated Datasets/disfold_Avengers_m.jsonl'
PATH_DS2 = './Mediated Datasets/disfold_fr_m.jsonl'
THRESHOLD = 0.99

In [264]:
# dfA = pd.read_json(PATH_DS1, lines=True)
# display(dfA)
# dfB = pd.read_json(PATH_DS2, lines=True)
# display(dfB)

In [265]:
def matches_list_index(path_ds1, path_ds2):
    pairs = []
    dfA = pd.read_json(path_ds1, lines=True)
    dfB = pd.read_json(path_ds2, lines=True)
    # set_index
    indexer = recordlinkage.Index()
    indexer.full()
    candidate_links  = indexer.index(dfA, dfB)
    # Comparison step
    compare_cl = recordlinkage.Compare()
    compare_cl.string("name", "name", method="jarowinkler", threshold=THRESHOLD, label="name")
    features = compare_cl.compute(candidate_links, dfA, dfB)
    # Classification step
    matches = features[features.sum(axis=1) >= 0.99]
    # display(matches)
    di = matches.to_dict()
    for pair in di['name'].keys():
        pairs.append(pair)
    return pairs

def row_by_index(path, i):
    df = pd.read_json(path, lines=True)
    return df.loc[i]

def rename_columns(columns_a, columns_b):
    columns_a = [c + '_l' for c in columns_a]
    columns_b = [c + '_r' for c in columns_b]
    # columns_join = columns_a + columns_b
    return columns_a, columns_b #, columns_join



In [266]:
pairs = matches_list_index(PATH_DS1, PATH_DS2)
print(len(pairs))
left = []
right = []
for pair in pairs:
    left.append(row_by_index(PATH_DS1, pair[0]))
    right.append(row_by_index(PATH_DS2, pair[1]))
df_left = pd.DataFrame(left)
df_right = pd.DataFrame(right)

display(df_left)
display(df_right)



899


,name,country,continent,founded,employees,ceo,market_cap,categories
0,apple,cupertino,ca,"april 1, 1976","100,000",timothy d cook,2825000000000,"unitedstates, technology, consumerelectronics,..."
1,danaher,washington,dc,1969,"78,000",rainer m blair,208030000000,"unitedstates, healthcare, diagnostics&research..."
2,merck,kenilworth,nj,1891,"67,000",robert m davis jd,210380000000,"unitedstates, healthcare, drugmanufacturersgen..."
3,bhp,melbourne,vic,1885,"40,110","mike p henry bsc, bsc chem",a$281.66 billion,"australia, basicmaterials, otherindustrialmeta..."
4,nike,beaverton,or,"january 25, 1964","73,300",john j donahoe ii,213600000000,"unitedstates, consumerdiscretionary, footwear&..."
...,...,...,...,...,...,...,...,...
894,teck resources limited,vancouver,bc,,"10,600","donald r lindsay bsc honours, mba",c$26.81 billion,"canada, basicmaterials, otherindustrialmetals&..."
895,sgs sa,geneva,switzerland,,"96,216",,chf19.83 billion,"switzerland, industrials, consultingservices, ..."
896,the saudi british bank,riyadh,saudiarabia,,"4,156",anthony william cripps,sr80.45 billion,"saudiarabia, financials, banks, saudiarabiafin..."
897,mitsui fudosan co ltd,tokyo,japan,,"23,992",masanobu komoda,2.629 trillion,"japan, realestate, realestatediversified, japa..."


,founded,employees,ceo,name
0,": april 1, 1976","employees: 100,000",ceo timothy d cook,apple
49,: 1969,"employees: 78,000",ceo rainer m blair,danaher
48,: 1891,"employees: 67,000",ceo robert m davis jd,merck
47,: 1885,"employees: 40,110","ceo mike p henry bsc, bsc chem",bhp
46,": january 25, 1964","employees: 73,300",ceo john j donahoe ii,nike
...,...,...,...,...
854,,"employees: 10,600","ceo donald r lindsay bsc honours, mba",teck resources limited
853,,"employees: 96,216",,sgs sa
852,,"employees: 4,156",ceo anthony william cripps,the saudi british bank
851,,"employees: 23,992",ceo masanobu komoda,mitsui fudosan co ltd


In [267]:
attr_cleaned = df_left.columns.values.tolist() + df_right.columns.values.tolist()
duplicates = [item for item, count in collections.Counter(attributes).items() if count > 1]
print(attr_cleaned)
print(duplicates)

['name', 'country', 'continent', 'founded', 'employees', 'ceo', 'market_cap', 'categories', 'founded', 'employees', 'ceo', 'name']
['name', 'founded', 'employees', 'ceo']


In [268]:
column_left, column_right = rename_columns(df_left.columns.values.tolist(), df_right.columns.values.tolist())

df_left.columns = column_left
df_right.columns = column_right

#pp = pd.DataFrame() 
#pp = pd.concat([df_left, df_right], axis=1)
#print(pp)

#joined_df = pd.concat([df_left, df_right], axis=1, ignore_index=True, sort=False)
#joined_df

ppp = pd.concat([df_left, df_right], axis=1)

# ppp = pd.merge(df_left, df_right, left_index=True, right_index=True)
display(ppp)


,name_l,country_l,continent_l,founded_l,employees_l,ceo_l,market_cap_l,categories_l,founded_r,employees_r,ceo_r,name_r
0,apple,cupertino,ca,"april 1, 1976","100,000",timothy d cook,2825000000000,"unitedstates, technology, consumerelectronics,...",": april 1, 1976","employees: 100,000",ceo timothy d cook,apple
1,danaher,washington,dc,1969,"78,000",rainer m blair,208030000000,"unitedstates, healthcare, diagnostics&research...",": april 4, 1975","employees: 181,000",ceo satya nadella,microsoft
2,merck,kenilworth,nj,1891,"67,000",robert m davis jd,210380000000,"unitedstates, healthcare, drugmanufacturersgen...",,,ceo amin h nasser,aramco
3,bhp,melbourne,vic,1885,"40,110","mike p henry bsc, bsc chem",a$281.66 billion,"australia, basicmaterials, otherindustrialmeta...",": october 2, 2015","employees: 156,500",ceo sundar pichai,alphabet
4,nike,beaverton,or,"january 25, 1964","73,300",john j donahoe ii,213600000000,"unitedstates, consumerdiscretionary, footwear&...",": july 5, 1994","employees: 1,608,000",ceo andrew r jassy,amazon
...,...,...,...,...,...,...,...,...,...,...,...,...
894,teck resources limited,vancouver,bc,,"10,600","donald r lindsay bsc honours, mba",c$26.81 billion,"canada, basicmaterials, otherindustrialmetals&...",: 1930,"employees: 20,300",ceo thomas sinnickson gayner,markel corporation
895,sgs sa,geneva,switzerland,,"96,216",,chf19.83 billion,"switzerland, industrials, consultingservices, ...",: 1939,"employees: 12,023","ceo j powell brown cpcu, cpcu","brown & brown, inc"
896,the saudi british bank,riyadh,saudiarabia,,"4,156",anthony william cripps,sr80.45 billion,"saudiarabia, financials, banks, saudiarabiafin...",,"employees: 16,196",ceo shingo konomoto,nomura research institute ltd
897,mitsui fudosan co ltd,tokyo,japan,,"23,992",masanobu komoda,2.629 trillion,"japan, realestate, realestatediversified, japa...",: 1986,"employees: 52,000",ceo william joseph hornbuckle iv,mgm resorts international


In [269]:
for i in duplicates:
    question = ppp[[i+'_l', i+'_r']].sample(20)
    labels = question.columns.values.tolist()
    display(question)
    scarta = int(input("Scegli la colonna da scartare "))
    ppp.drop(labels[scarta], axis=1, inplace=True)
    #print(labels[scarta])
    
display(ppp)

,name_l,name_r
231,ubs group ag,industrial bank co ltd
802,"monolithic power syste, inc",panasonic corp
65,union pacific,texas instruments
286,the saudi national bank,enterprise products partners lp
48,aramco,merck
29,roche,asml
742,"mid-america apartment communities, inc","slack technologies, inc"
195,the southern company,"intercontinental exchange, inc"
461,lyondellbasell industries nv,kuaishou technology
619,tongwei co ltd,spotify technology sa


Scegli la colonna da scartare 0


,founded_l,founded_r
645,"august 27, 1906",": july 29, 2008"
334,,": minneapolis, minnesota, u.s.1902"
565,,
680,,
68,"may 15, 1940",: 21 june 1879
567,,: 1883
412,"january 4, 2011",: 1935
891,1958,
647,30 november 1999,
743,"vancouver, british columbia, canada 2009",: 1977


Scegli la colonna da scartare 0


,employees_l,employees_r
260,"86,700","employees: 2,180"
764,,"employees: 118,400"
131,"69,205","employees: 292,067"
310,"47,000","employees: 36,600"
52,"33,400","employees: 83,100"
166,"89,782","employees: 89,782"
69,"373,814","employees: 15,000"
476,"37,000","employees: 43,257"
854,"20,300","employees: 10,600"
245,"24,700","employees: 41,000"


Scegli la colonna da scartare 1


,ceo_l,ceo_r
223,e scott santi,ceo werner baumann
226,werner baumann,ceo e scott santi
278,dara khosrowshahi,ceo aneel bhusri
896,anthony william cripps,ceo shingo konomoto
390,thierry delaporte,ceo william c rhodes iii
347,dr osamu okuda,ceo sarah m london
810,,
440,,ceo jeffrey s sloan jd
527,"ola rollan bsc, bsc econ",ceo roger crosby hochschild
831,,


Scegli la colonna da scartare 1


,country_l,continent_l,employees_l,ceo_l,market_cap_l,categories_l,founded_r,name_r
0,cupertino,ca,"100,000",timothy d cook,2825000000000,"unitedstates, technology, consumerelectronics,...",": april 1, 1976",apple
1,washington,dc,"78,000",rainer m blair,208030000000,"unitedstates, healthcare, diagnostics&research...",": april 4, 1975",microsoft
2,kenilworth,nj,"67,000",robert m davis jd,210380000000,"unitedstates, healthcare, drugmanufacturersgen...",,aramco
3,melbourne,vic,"40,110","mike p henry bsc, bsc chem",a$281.66 billion,"australia, basicmaterials, otherindustrialmeta...",": october 2, 2015",alphabet
4,beaverton,or,"73,300",john j donahoe ii,213600000000,"unitedstates, consumerdiscretionary, footwear&...",": july 5, 1994",amazon
...,...,...,...,...,...,...,...,...
894,vancouver,bc,"10,600","donald r lindsay bsc honours, mba",c$26.81 billion,"canada, basicmaterials, otherindustrialmetals&...",: 1930,markel corporation
895,geneva,switzerland,"96,216",,chf19.83 billion,"switzerland, industrials, consultingservices, ...",: 1939,"brown & brown, inc"
896,riyadh,saudiarabia,"4,156",anthony william cripps,sr80.45 billion,"saudiarabia, financials, banks, saudiarabiafin...",,nomura research institute ltd
897,tokyo,japan,"23,992",masanobu komoda,2.629 trillion,"japan, realestate, realestatediversified, japa...",: 1986,mgm resorts international


In [ ]:
# 1) PRENDERE RECORDS CHE NON MATCHANO DAL PRIMO DATASETS
# 2) PRENDERE RECORDS CHE NON MATCHANO DAL SECONDO DATASETS
# 3) PRENDERE I RECORD CHE MATCHANO (ppp) 
# 4) AGGIUNGERE TUTTO A UN FILE JSONL


In [200]:
df_left = df_left.reset_index(drop=True)
display(df_left)

df_right = df_right.reset_index(drop=True)
display(df_right)

ppp = pd.concat([df_left, df_right], axis=1)

# ppp = pd.merge(df_left, df_right, left_index=True, right_index=True)
display(ppp)


# bisogna ricambiare il nome degli attributi

,name,country,continent,founded,employees,ceo,market_cap,categories
0,apple,cupertino,ca,"april 1, 1976","100,000",timothy d cook,2825000000000,"unitedstates, technology, consumerelectronics,..."
1,danaher,washington,dc,1969,"78,000",rainer m blair,208030000000,"unitedstates, healthcare, diagnostics&research..."
2,merck,kenilworth,nj,1891,"67,000",robert m davis jd,210380000000,"unitedstates, healthcare, drugmanufacturersgen..."
3,bhp,melbourne,vic,1885,"40,110","mike p henry bsc, bsc chem",a$281.66 billion,"australia, basicmaterials, otherindustrialmeta..."
4,nike,beaverton,or,"january 25, 1964","73,300",john j donahoe ii,213600000000,"unitedstates, consumerdiscretionary, footwear&..."
...,...,...,...,...,...,...,...,...
894,teck resources limited,vancouver,bc,,"10,600","donald r lindsay bsc honours, mba",c$26.81 billion,"canada, basicmaterials, otherindustrialmetals&..."
895,sgs sa,geneva,switzerland,,"96,216",,chf19.83 billion,"switzerland, industrials, consultingservices, ..."
896,the saudi british bank,riyadh,saudiarabia,,"4,156",anthony william cripps,sr80.45 billion,"saudiarabia, financials, banks, saudiarabiafin..."
897,mitsui fudosan co ltd,tokyo,japan,,"23,992",masanobu komoda,2.629 trillion,"japan, realestate, realestatediversified, japa..."


,founded,employees,ceo,name
0,": april 1, 1976","employees: 100,000",ceo timothy d cook,apple
1,: 1969,"employees: 78,000",ceo rainer m blair,danaher
2,: 1891,"employees: 67,000",ceo robert m davis jd,merck
3,: 1885,"employees: 40,110","ceo mike p henry bsc, bsc chem",bhp
4,": january 25, 1964","employees: 73,300",ceo john j donahoe ii,nike
...,...,...,...,...
894,,"employees: 10,600","ceo donald r lindsay bsc honours, mba",teck resources limited
895,,"employees: 96,216",,sgs sa
896,,"employees: 4,156",ceo anthony william cripps,the saudi british bank
897,,"employees: 23,992",ceo masanobu komoda,mitsui fudosan co ltd


,name,country,continent,founded,employees,ceo,market_cap,categories,founded,employees,ceo,name
0,apple,cupertino,ca,"april 1, 1976","100,000",timothy d cook,2825000000000,"unitedstates, technology, consumerelectronics,...",": april 1, 1976","employees: 100,000",ceo timothy d cook,apple
1,danaher,washington,dc,1969,"78,000",rainer m blair,208030000000,"unitedstates, healthcare, diagnostics&research...",: 1969,"employees: 78,000",ceo rainer m blair,danaher
2,merck,kenilworth,nj,1891,"67,000",robert m davis jd,210380000000,"unitedstates, healthcare, drugmanufacturersgen...",: 1891,"employees: 67,000",ceo robert m davis jd,merck
3,bhp,melbourne,vic,1885,"40,110","mike p henry bsc, bsc chem",a$281.66 billion,"australia, basicmaterials, otherindustrialmeta...",: 1885,"employees: 40,110","ceo mike p henry bsc, bsc chem",bhp
4,nike,beaverton,or,"january 25, 1964","73,300",john j donahoe ii,213600000000,"unitedstates, consumerdiscretionary, footwear&...",": january 25, 1964","employees: 73,300",ceo john j donahoe ii,nike
...,...,...,...,...,...,...,...,...,...,...,...,...
894,teck resources limited,vancouver,bc,,"10,600","donald r lindsay bsc honours, mba",c$26.81 billion,"canada, basicmaterials, otherindustrialmetals&...",,"employees: 10,600","ceo donald r lindsay bsc honours, mba",teck resources limited
895,sgs sa,geneva,switzerland,,"96,216",,chf19.83 billion,"switzerland, industrials, consultingservices, ...",,"employees: 96,216",,sgs sa
896,the saudi british bank,riyadh,saudiarabia,,"4,156",anthony william cripps,sr80.45 billion,"saudiarabia, financials, banks, saudiarabiafin...",,"employees: 4,156",ceo anthony william cripps,the saudi british bank
897,mitsui fudosan co ltd,tokyo,japan,,"23,992",masanobu komoda,2.629 trillion,"japan, realestate, realestatediversified, japa...",,"employees: 23,992",ceo masanobu komoda,mitsui fudosan co ltd
